# TODO:
1.   Distort images: (gritty, dark, noise) using imgaug
2.   Retrain on theese images
3.   Test the old CNN agaisnt the new ones on theese images

## this is done and this is the results

### weakest model trained on new images:

{'fscore': 0.8493919550982226, 'precision': 0.8598484848484849, 'recall': 0.8391866913123844}
Average prediction time:5.35787731768137 ms

### less weak model trained on new images: 

{'fscore': 0.9339449541284405, 'precision': 0.9271402550091075, 'recall': 0.9408502772643254}
Average prediction time:5.5791603992263 ms

### weakest model trained on normal images:

{'fscore': 0.5445462114904247, 'precision': 0.4954545454545455, 'recall': 0.6044362292051756}
Average prediction time:5.511541443177495 ms

### strongest model trained on normal images:

{'fscore': 0.5556831228473019, 'precision': 0.7333333333333333, 'recall': 0.44731977818853974}
Average prediction time:6.131902755982426 ms


# IMPORTANT: theese are old values.
#### New lego characters are now beeing trained on (the ones we will actually look for in the end). However the old model was able to detect them without beeing trained on them. With about 60-70% precision. Altough the stronger model was needed, and it missread some objects as lego characters.



In [2]:
!nvidia-smi # You want 16280MiB, just hardreset runtime and run again.

Sat Sep 26 06:55:54 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.66       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    25W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Initialization

In [ ]:
%tensorflow_version 1.x

from google.colab import drive
drive.mount('/content/drive')
!mkdir /content/drive/My\ Drive/ModelsH5

!mkdir /content/GeneratedDataset
!mkdir /content/GeneratedDataset/GeneratedImages
!mkdir /content/GeneratedDataset/GeneratedImagesXml
!mkdir /content/GeneratedDataset/GeneratedImages_validation
!mkdir /content/GeneratedDataset/GeneratedImagesXml_validation

!wget https://www.robots.ox.ac.uk/~vgg/data/dtd/download/dtd-r1.0.1.tar.gz
!tar xf dtd-r1.0.1.tar.gz
!rm /content/dtd-r1.0.1.tar.gz

!git clone https://github.com/AIWintermuteAI/aXeleRate.git
!pip uninstall -y imgaug && pip uninstall -y albumentations && pip install imgaug==0.4

%cd /content/
!mkdir /content/Yolo-digit-detector
%cd /content/Yolo-digit-detector
!git clone https://github.com/abbsimoga/MDH-samarbetande-robotar

In [ ]:
%cd /content
from keras import backend as K
import sys
sys.path.append('/content/aXeleRate')
from axelerate import setup_training, setup_inference
import os

actual_epoch = 20
learning_rate = 1e-4

In [5]:
def get_model_path(model_path_txt):
    f = open(model_path_txt, "r")
    model_path = f.read()
    f.close()
    return model_path

def get_config(model_path, architecture, batch_size, saved_folder, actual_epoch, learning_rate):
    config = {
            "model":{
                "type":             "Detector",
                "architecture":     architecture,
                "input_size":       224,
                "anchors":          [0.57273, 0.677385, 1.87446, 2.06253, 3.33843, 5.47434, 7.88282, 3.52778, 9.77052, 9.16828],
                "labels":           ["lego gubbe"],
                "coord_scale" : 		1.0,
                "class_scale" : 		1.0,
                "object_scale" : 		5.0,
                "no_object_scale" : 1.0
            },
            "weights" : {
                "full":   				  model_path,
                "backend":   		    "imagenet"
            },
            "train" : {
                "actual_epoch":         actual_epoch,
                "train_image_folder":   "/content/GeneratedDataset/GeneratedImages",
                "train_annot_folder":   "/content/GeneratedDataset/GeneratedImagesXml",
                "train_times":          1,
                "valid_image_folder":   "/content/GeneratedDataset/GeneratedImages_validation",
                "valid_annot_folder":   "/content/GeneratedDataset/GeneratedImagesXml_validation",
                "valid_times":          1,
                "valid_metric":         "mAP",
                "batch_size":           batch_size,
                "learning_rate":        learning_rate,
                "saved_folder":   	    saved_folder,
                "first_trainable_layer":"",
                "augumentation":				False,
                "is_only_detect" : 		  False
            },
            "converter" : {
                "type":   				["k210","kmodel"]
            }
    }

    return config

In [6]:
# utilizing threading:
# time: 40.67213749885559 per1k: 40.672142028808594 per1: 0.040672143697738646


# utilizing gpu:
# time: 37.83721184376 per1k: 37.83721184376 per1: 0.03783721184376001


# utilizing multiproccesing:
# time: 19.642900943756104 per1k: 19.642902135849 per1: 0.019642902851104736


# somthing changes inbetween this value and the next and i cant figure out what, its a lot slower now
# expected time increese (around 8 seconds), but it jumped to:


# using augmentation the time increeses significantly to around:
# time: 33.92477869987488 per1k: 33.924779415130615 per1: 0.03392478013038635


### IMPORTANT:
# this is an old way that i created images to train on, a way better one i did with card detection can be found @:
# https://drive.google.com/file/d/1AF1ElrFIrhH1V4A_hkdC7oedH2chb5vZ/view?usp=sharing
# but couldnt bother to change this one, since heavy time restriction


minSize = 28
maxSize = 112
maxCharactersAllowed = 6

import cv2
import imutils
import os
import random
import numpy as np
import xml.etree.cElementTree as ET
from glob import glob
import time
import timeit
import concurrent.futures
import imgaug.augmenters as iaa

seq = iaa.Sequential([
    iaa.Add((-40, 15), per_channel=0.5),
    iaa.AdditiveGaussianNoise(scale=(0, 0.10*255), per_channel=True),
    iaa.CoarseSaltAndPepper(0.025, size_percent=(0.01, 0.08), per_channel=True),
    iaa.JpegCompression(compression=(10, 35))
])

def create_root(file_prefix, width, height):
    root = ET.Element("annotations")
    ET.SubElement(root, "filename").text = "{}.JPEG".format(file_prefix)
    ET.SubElement(root, "folder").text = "images"
    size = ET.SubElement(root, "size")
    ET.SubElement(size, "width").text = str(width)
    ET.SubElement(size, "height").text = str(height)
    ET.SubElement(size, "depth").text = "3"
    return root
 
def create_object_annotation(root, voc_labels):
    for voc_label in voc_labels:
        obj = ET.SubElement(root, "object")
        ET.SubElement(obj, "name").text = voc_label[0]
        ET.SubElement(obj, "pose").text = "Unspecified"
        ET.SubElement(obj, "truncated").text = str(0)
        ET.SubElement(obj, "difficult").text = str(0)
        bbox = ET.SubElement(obj, "bndbox")
        ET.SubElement(bbox, "xmin").text = str(voc_label[1])
        ET.SubElement(bbox, "ymin").text = str(voc_label[2])
        ET.SubElement(bbox, "xmax").text = str(voc_label[3])
        ET.SubElement(bbox, "ymax").text = str(voc_label[4])
    return root

def create_images(itterations): 
    if itterations % 10000 == 0:
        print("Current images:",itterations)
    charactersOnCanvas = []
    resizedBackground=cv2.resize(cv2.cvtColor(cv2.imread(random.choice(glob("/content/dtd/images/*/*.jpg"))), cv2.COLOR_RGB2RGBA),(224,224))

    charactersAmount = random.randint(1, maxCharactersAllowed)
    for amounts in range(charactersAmount):
        skip = False
        characterDir = "/content/Yolo-digit-detector/MDH-samarbetande-robotar/OpenMV/YoloObjDetection/lego-gubbar-detection/Edit Characters/FramesNoBackground/"+random.choice(os.listdir("/content/Yolo-digit-detector/MDH-samarbetande-robotar/OpenMV/YoloObjDetection/lego-gubbar-detection/Edit Characters/FramesNoBackground"))
        imgCharacter = cv2.imread(characterDir, cv2.IMREAD_UNCHANGED)

        rotation = random.randint(0,360)
        rotatedCharacter = imutils.rotate_bound(imgCharacter, rotation)

        largeSide = max([rotatedCharacter.shape[1], rotatedCharacter.shape[0]])
        maxScale = int((maxSize/largeSide)*100)
        minScale = int((minSize/largeSide)*100)
        scalePercent = random.randint(minScale, maxScale)
        resizedCharacter = cv2.resize(rotatedCharacter, (int(rotatedCharacter.shape[1]*scalePercent/100),int(rotatedCharacter.shape[0]*scalePercent/100)))

        y_offset = random.randint(0, resizedBackground.shape[0]-resizedCharacter.shape[0])
        x_offset = random.randint(0, resizedBackground.shape[1]-resizedCharacter.shape[1])
        y1, y2 = y_offset, y_offset + resizedCharacter.shape[0]
        x1, x2 = x_offset, x_offset + resizedCharacter.shape[1]

        alpha_s = resizedCharacter[:, :, 3] / 255.0
        alpha_l = 1.0 - alpha_s

        if charactersOnCanvas:
            for _, x1_, y1_, x2_, y2_ in charactersOnCanvas:
                if x2_ < x1 or x2 < x1_ or y2_ < y1 or y2 < y1_:
                    continue                
                skip=True
                break
            
        if skip:
            continue

        for c in range(0, 3):
            resizedBackground[y1:y2, x1:x2, c] = (alpha_s * resizedCharacter[:, :, c] + alpha_l * resizedBackground[y1:y2, x1:x2, c])

        charactersOnCanvas.append(["lego gubbe", x1, y1, x2, y2])

    file_prefix = str(itterations).zfill(8)
    root = create_root(file_prefix, resizedBackground.shape[0], resizedBackground.shape[1])
    root = create_object_annotation(root, charactersOnCanvas)
    tree = ET.ElementTree(root) 
    
    tree.write("/content/GeneratedDataset/GeneratedImagesXml{}/{}.xml".format("_validation" if validation else "", file_prefix))
    # cv2.imwrite("/content/GeneratedDataset/GeneratedImages{}/{}.JPEG".format("_validation" if validation else "", file_prefix), resizedBackground)
    cv2.imwrite("/content/GeneratedDataset/GeneratedImages{}/{}.JPEG".format("_validation" if validation else "", file_prefix), seq(image=resizedBackground[:, :, :3]))

# Select model

#### Possible models

##### Any model bigger than MobileNet5_0 usualy crashes when paired with other algorythms like, findblob/lineregression

- Full Yolo
- Tiny Yolo
- MobileNet1_0
- MobileNet7_5
- MobileNet5_0 (Recommended) slower better results
- MobileNet2_5 (Recommended) faster worse results
- SqueezeNet
- NASNetMobile
- DenseNet121
- ResNet50

#### Exact training values of each model can be found underneath

### MobileNet2_5
*   Average runtime: 5.078327224915286 ms
*   Fscore:          0.9118483412322275
*   Precision:       0.9232245681381958
*   Recall:          0.900749063670412
*   Model size:      226,254

In [ ]:
!mkdir /content/drive/My\ Drive/ModelsH5/MobileNet2_5

architecture = "MobileNet2_5"
batch_size = 384

saved_folder = "/content/drive/My Drive/ModelsH5/MobileNet2_5"
model_path_txt = "/content/drive/My Drive/ModelsH5/MobileNet2_5/model_path.txt"

if not os.path.isfile(model_path_txt):
    f = open(model_path_txt,"w+")
    f.close()

### MobileNet5_0

*   Average runtime: 5.546079581999875 ms
*   Fscore:          0.976657329598506
*   Precision:       0.9739292364990689
*   Recall:          0.9794007490636704
*   Model size:      844 926

In [7]:
!mkdir /content/drive/My\ Drive/ModelsH5/MobileNet5_0

architecture = "MobileNet5_0"
batch_size = 192

saved_folder = "/content/drive/My Drive/ModelsH5/MobileNet5_0"
model_path_txt = "/content/drive/My Drive/ModelsH5/MobileNet5_0/model_path.txt"

if not os.path.isfile(model_path_txt):
    f = open(model_path_txt,"w+")
    f.close()

### MobileNet7_5
*   Average runtime: 6.327065119302895 ms
*   Fscore:          0.9804287045666357
*   Precision:       0.9758812615955473
*   Recall:          0.9850187265917603
*   Model size:      1 856 046

In [ ]:
!mkdir /content/drive/My\ Drive/ModelsH5/MobileNet7_5

architecture = "MobileNet7_5"
batch_size = 192

saved_folder = "/content/drive/My Drive/ModelsH5/MobileNet7_5"
model_path_txt = "/content/drive/My Drive/ModelsH5/MobileNet7_5/model_path.txt"

if not os.path.isfile(model_path_txt):
    f = open(model_path_txt,"w+")
    f.close()

### MobileNet1_0
*   Average runtime: 7.210679801113634 ms
*   Fscore:          0.9915651358950328
*   Precision:       0.9924953095684803
*   Recall:          0.9906367041198502
*   Model size:      3 259 614

In [ ]:
!mkdir /content/drive/My\ Drive/ModelsH5/MobileNet1_0

architecture = "MobileNet1_0"
batch_size = 192

saved_folder = "/content/drive/My Drive/ModelsH5/MobileNet1_0"
model_path_txt = "/content/drive/My Drive/ModelsH5/MobileNet1_0/model_path.txt"

if not os.path.isfile(model_path_txt):
    f = open(model_path_txt,"w+")
    f.close()

#  Train

### IMPORTANT:
#### This is not 100% set time, its an aproximation
#### expect to wait LONGER for training and to create the dataset 

Creating 100 000 images takes 45 min

Training time depends on model but can aproximate to 175 min on large model, 135 on MobileNet2_5

### Train 1 time
#### (recommended)

In [ ]:
actual_epoch = 20
validation = False
start_time = time.time()
pool = concurrent.futures.ProcessPoolExecutor(max_workers=None)
pool.map(create_images, range(150000))
pool.shutdown(wait=True)
print("time:",time.time()-start_time,"per1k:",(time.time()-start_time)/100,"per1:",(time.time()-start_time)/100000)

validation = True
start_time = time.time()
pool = concurrent.futures.ProcessPoolExecutor(max_workers=None)
pool.map(create_images, range(1000))
pool.shutdown(wait=True)
print("time:",time.time()-start_time,"per1k:",(time.time()-start_time)/1,"per1:",(time.time()-start_time)/1000)

model_path = get_model_path(model_path_txt)
config = get_config(model_path, architecture, batch_size, saved_folder, actual_epoch, learning_rate)

K.clear_session()
model_path = setup_training(config_dict=config)

f = open(model_path_txt, "w")
f.write(model_path)
f.close()

### Train to your heart's content
#### creates a smaller dataset and trains for fewer epochs to be able to change models learning rate inbetween.

### IMPORTANT
#### this loop will never break but your model will be saved after each itteration

In [ ]:
actual_epoch = 5

while True:
    validation = False
    start_time = time.time()
    pool = concurrent.futures.ProcessPoolExecutor(max_workers=None)
    pool.map(create_images, range(50000))
    pool.shutdown(wait=True)
    print("time:",time.time()-start_time,"per1k:",(time.time()-start_time)/50,"per1:",(time.time()-start_time)/50000)

    validation = True
    start_time = time.time()
    pool = concurrent.futures.ProcessPoolExecutor(max_workers=None)
    pool.map(create_images, range(1000))
    pool.shutdown(wait=True)
    print("time:",time.time()-start_time,"per1k:",(time.time()-start_time)/1,"per1:",(time.time()-start_time)/1000)


    model_path = get_model_path(model_path_txt)
    config = get_config(model_path, architecture, batch_size, saved_folder, actual_epoch, learning_rate)

    K.clear_session()
    model_path = setup_training(config_dict=config)

    f = open(model_path_txt, "w")
    f.write(model_path)
    f.close()

    learning_rate = learning_rate/1.5

# Convert

In [ ]:
f = open(model_path_txt, "r")
model_path = f.read()
f.close()

validation = True
start_time = time.time()
pool = concurrent.futures.ProcessPoolExecutor(max_workers=None)
pool.map(create_images, range(1000))
pool.shutdown(wait=True)
print("time:",time.time()-start_time,"per1k:",(time.time()-start_time)/1,"per1:",(time.time()-start_time)/1000)

%cd /content
!git clone https://github.com/sipeed/Maix_Toolbox
%cd /content/Maix_Toolbox
!bash get_nncase.sh
%cd /content/Maix_Toolbox/ncc
!tar -xJf /content/Maix_Toolbox/ncc/ncc-linux-x86_64.tar.xz
!rm ncc-linux-x86_64.tar.xz

%cd /content/Maix_Toolbox 

!cp -a /content/GeneratedDataset/GeneratedImages_validation/. /content/Maix_Toolbox/images/

!rm tflite2kmodel.sh
!wget https://raw.githubusercontent.com/abbjoafli/ComputerVision/master/files/tflite2kmodel.sh

!cp {"/content/drive/My\ Drive/ModelsH5/"+model_path[33:-17]+"/YOLO_best_mAP.tflite"} /content/Maix_Toolbox/

!bash tflite2kmodel.sh /content/Maix_Toolbox/YOLO_best_mAP.tflite /content/Maix_Toolbox/YOLO_best_mAP.kmodel -i tflite -o kmodel -t k210 --dataset /content/Maix_Toolbox/images

!cp /content/Maix_Toolbox/YOLO_best_mAP.kmodel {"/content/drive/My\ Drive/ModelsH5/"+model_path[33:-17]}


# Evaluate

In [ ]:
validation = True
start_time = time.time()
pool = concurrent.futures.ProcessPoolExecutor(max_workers=None)
pool.map(create_images, range(250))
pool.shutdown(wait=True)
print("time:",time.time()-start_time,"per1k:",(time.time()-start_time)/.25,"per1:",(time.time()-start_time)/250)

model_path = get_model_path(model_path_txt)
config = get_config(model_path, architecture, batch_size, saved_folder, actual_epoch, learning_rate)

K.clear_session()
setup_inference(config, model_path)